In [2]:
"""
This notebook processes a small set of emails using a
linear SVM and evaluates its performance on a test set
"""
import numpy as np
import pandas as pd

import scipy.io as sio
from sklearn import svm, datasets

import re
import pprint
from stemming.porter2 import stem

np.set_printoptions(precision=3)
ansi = {'underline': '\033[4m', 'bold': '\033[1m', 'end':'\033[0m'}

In [3]:
def processEmail(email, vocab):
    """Runs an email through a set of regexes, stemming, and maps every word to the vocab set"""
    
    # Compile regexes
    html_regex = re.compile('<[^<>]+>') # Strip html
    num_regex = re.compile('[0-9]+')
    url_regex = re.compile(r'(http|https)://[^\s]*')
    email_regex = re.compile('[^\s]+@[^\s]+')
    dollar_regex = re.compile('[$]+')
    
    email = email.lower()
    email = email.replace('\n', ' ')
    
    email = html_regex.sub(' ', email)
    email = num_regex.sub('number', email)
    email = url_regex.sub('httpaddr', email)
    email = email_regex.sub('emailaddr', email)
    email = dollar_regex.sub('dollar', email)
    
    # Remove punctuation
    for c in r"""@$/#.-:&*+=[]?!(){},''">_<;√""":
        email = email.replace(c, ' ')
    
    email = re.sub('\s+', ' ', email).strip() # delete extra spaces and strip
    email = email.split(' ') # split by spaces
    email = [stem(x) for x in email] # stem words in email
    
    word_indices = []
    length = 0 # for printing
    for word in email:
        index = vocab[vocab['words'] == word].index.tolist()
        if index:
            word_indices.append(index[0])
        
        # Printing
        if length + len(word) + 1 > 78:
            print()
            length = 0
        print('{} '.format(word), end='')
        length += len(word) + 1
    print('\n')
    return word_indices

In [4]:
def emailFeatures(word_indices, vocab):
    """Maps every index in word_indices to its corresponding feature"""
    return [1 if index in word_indices else 0 for index in range(len(vocab))]

In [5]:
email_1 = open('emailSample1.txt', 'r').read()
vocab = pd.read_table('vocab.txt',usecols=[1], names=['words'])
pp = pprint.PrettyPrinter(compact=True)
vocab.head()

words
0     aa
1     ab
2   abil
3    abl
4  about

In [6]:
index_list = processEmail(email_1, vocab)
features = emailFeatures(index_list, vocab)
pp.pprint(index_list)

anyon know how much it cost to host a web portal well it depend on how mani 
visitor you re expect this can be anywher from less than number buck a month 
to a coupl of dollarnumb you should checkout httpaddr or perhap amazon ecnumb 
if your run someth big to unsubscrib yourself from this mail list send an 
email to emailaddr 

[85, 915, 793, 1076, 882, 369, 1698, 789, 1821, 1830, 882, 430, 1170, 793, 1001,
 1892, 1363, 591, 237, 161, 88, 687, 944, 1662, 1119, 1061, 1698, 374, 1161,
 478, 1892, 1509, 798, 1181, 1236, 809, 1894, 1439, 1546, 180, 1698, 1757, 1895,
 687, 991, 960, 1476, 70, 529, 1698, 530]


In [7]:
# Linear SVM for spam classification
data = sio.loadmat('spamTrain.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [8]:
X = data['X']
y = data['y'].flatten()

C = 0.1

In [9]:
model = svm.LinearSVC(C=C)
model.fit(X, y.flatten())
pred = model.predict(X)

accuracy = np.mean(pred == y) * 100
print('Training accuracy: {0:0.3g}'.format(accuracy))

Training accuracy: 100


In [10]:
# Spam testing set

data = sio.loadmat('spamTest.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Xtest', 'ytest'])

In [11]:
Xtest = data['Xtest']
ytest = data['ytest'].flatten()

In [12]:
pred = model.predict(Xtest)
accuracy = np.mean(pred == ytest) * 100
print('Testing accuracy: {0:0.3g}'.format(accuracy))

Testing accuracy: 99.2


In [13]:
weights = model.coef_.flatten()
sorted_indices = weights.argsort()[::-1][:15] # reverse sorted arguments

top_spam = [[vocab['words'][index], weights[index]] for index in sorted_indices]

print('\n * Top predictors of spam * \n')
print('[Word, weight]')
for item in top_spam: print(item)


 * Top predictors of spam * 

[Word, weight]
['our', 0.42166483953440498]
['remov', 0.38717315255196288]
['click', 0.38705980092527098]
['basenumb', 0.34661724232825103]
['guarante', 0.34168552089260801]
['visit', 0.30302786782012386]
['bodi', 0.26352348471222414]
['will', 0.24439379638860914]
['numberb', 0.23879490843331866]
['price', 0.23419904572386757]
['dollar', 0.23231478206570594]
['nbsp', 0.22708087250962239]
['below', 0.22319898687307405]
['lo', 0.21999371144492108]
['most', 0.21454837016614811]
